--------------------------------------------------------------------------------
Calculate the final concentration using diffuse concentration boundary
--------------------------------------------------------------------------------

Example to obtain the final concentration at well using a recharge inflicted contamination as source. The particle are added 1m below groundlevel adding a single particle per column.

________________________________________

Import packages
-----------------

First we import the necessary python packages

In [23]:
import pandas as pd
import datetime as dt
from pathlib import Path
from set_cwd_to_project_root import project_root

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import os
from pandas import read_csv
from pandas import read_excel
import math
from scipy.special import kn as besselk
import sutra2.Analytical_Well as AW
import sutra2.ModPath_Well as mpw
import sutra2.Transport_Removal as TR

import warnings 
warnings.filterwarnings(action= 'ignore')

# get directory of this file
path = Path(project_root)
print(path)

%load_ext autoreload
%autoreload 2

d:\Sutra2_tool\sutra2\research
The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


Scenario with feces at surface level near pumping well.
Case: phreatic
organism_name: "MS2" 
Vadose zone: False
redox: anoxic
pH: neutral (7.5)
semiconfined scheme with gravelpack (but with recharge): modpath run.

In [24]:
# organism name
organism_name = "MS2"
# well discharge
well_discharge = -1000.
# recharge_rate
recharge_rate = 0.001
# model radius [m]
model_radius = 564. 
# model_radius = math.sqrt(abs(well_discharge / (math.pi * recharge_rate)))

# microbial removal properties
# organism_name = 'MS2'
alpha0 = {"suboxic": 1.e-3, "anoxic": 1.e-5, "deeply_anoxic": 1.e-5}
pH0 = {"suboxic": 6.6, "anoxic": 6.8, "deeply_anoxic": 6.8}
organism_diam =  2.33e-8
mu1 = {"suboxic": 0.039,"anoxic": 0.023,"deeply_anoxic": 0.023}

removal_parameters = {organism_name: 
                        {"organism_name": organism_name,
                            "alpha0": alpha0,
                            "pH0": pH0,
                            "organism_diam": organism_diam,
                            "mu1": mu1
                        }
                    }
# Removal parameters organism
rem_parms = removal_parameters[organism_name]

# Scenario runs
df_scen_path = os.path.join(path.parent ,"testing","scenarios_diffuse_sensitivity_mpw_220811.xlsx")
df_scen = pd.read_excel(df_scen_path, sheet_name = "scenarios_diffuse_sensitivity")

# df_output
df_output = df_scen
df_output["C_final"] = None

In [25]:
# sensitivity scenario runs
for iScen in df_scen.Scen_nr.values:

    if iScen != 1:
        continue
    # Vary number of columns and layers
    ncols_near_well = df_scen.loc[df_scen.Scen_nr == iScen,"ncols_near_well"].values[0]
    ncols_far_well = df_scen.loc[df_scen.Scen_nr == iScen,"ncols_far_well"].values[0]
    nlayers_shallow_aquifer = df_scen.loc[df_scen.Scen_nr == iScen,"nlayers_shallow_aquifer"].values[0]
    nlayers_target_aquifer = df_scen.loc[df_scen.Scen_nr == iScen,"nlayers_target_aquifer"].values[0]

    # workspace
    workspace = os.path.join(path.parent,"testing", "test_mpw_mbo_defecation_human_noleak_diffuse_sensitivity")
    if not os.path.exists(workspace):
        os.makedirs(workspace)
    print(workspace)
    
    # Hydrochem Schematisation
    test_feces_contamination = AW.HydroChemicalSchematisation(schematisation_type='semiconfined',
                                computation_method = 'modpath',
                                what_to_export='all',
                                removal_function = 'mbo',
                                # biodegradation_sorbed_phase = False,
                                well_discharge=well_discharge,
                                # vertical_resistance_shallow_aquifer=500,
                                porosity_vadose_zone=0.33,
                                porosity_shallow_aquifer=0.33,
                                porosity_target_aquifer=0.33,
                                porosity_gravelpack=0.33,
                                porosity_clayseal=0.33,
                                recharge_rate=recharge_rate,
                                moisture_content_vadose_zone=0.15,
                                ground_surface = 0.0,
                                thickness_vadose_zone_at_boundary=0.,
                                thickness_shallow_aquifer=30.,
                                thickness_target_aquifer=20.,
                                hor_permeability_target_aquifer= 10.0,
                                hor_permeability_shallow_aquifer = 10.,
                                hor_permeability_gravelpack= 1000.,
                                hor_permeability_clayseal= 0.005,
                                vertical_anisotropy_shallow_aquifer = 5.,
                                vertical_anisotropy_target_aquifer = 5.,
                                vertical_anisotropy_gravelpack = 1.,
                                vertical_anisotropy_clayseal = 1.,
                                thickness_full_capillary_fringe=0.,
                                grainsize_vadose_zone= 0.00025,
                                grainsize_shallow_aquifer=0.00025,
                                grainsize_target_aquifer=0.00025,
                                redox_vadose_zone='anoxic', 
                                redox_shallow_aquifer='anoxic',
                                redox_target_aquifer='anoxic',
                                pH_vadose_zone=7.5,
                                pH_shallow_aquifer=7.5,
                                pH_target_aquifer=7.5,
                                temp_water=12.,
                                solid_density_vadose_zone= 2.650, 
                                solid_density_shallow_aquifer= 2.650, 
                                solid_density_target_aquifer= 2.650, 
                                diameter_borehole = 0.75,
                                name = organism_name,
                                diameter_filterscreen = 0.2,
                                diameter_gravelpack = 0.75,
                                diameter_clayseal = 0.75,
                                point_input_concentration = 1.,
                                diffuse_input_concentration=2.15E5,
                                discharge_point_contamination = 100.,#made up value
                                top_clayseal = 0,
                                bottom_clayseal = -1.,
                                top_gravelpack = -1.,
                                grainsize_gravelpack=0.001,
                                grainsize_clayseal=0.000001,

                                compute_contamination_for_date=dt.datetime.strptime('2020-01-01',"%Y-%m-%d"),
                                # Modpath grid parms
                                ncols_near_well = ncols_near_well,
                                ncols_far_well = ncols_far_well,
                                nlayers_shallow_aquifer = nlayers_shallow_aquifer,
                                nlayers_target_aquifer = nlayers_target_aquifer,
                                model_radius = model_radius
                            )

    test_feces_contamination.make_dictionary()

    # Create point_parameters (i.e. starting points) from scratch
    point_parameters = {}
    # Copy point_parameters input
    test_feces_contamination.point_parameters = point_parameters

    # # Remove/empty concentration_boundary_parameters
    # test_feces_contamination.concentration_boundary_parameters = {}


    # semiconfined, but with recharge instead of constant head
    recharge_parameters = {
                        'source1': {
                            'recharge': recharge_rate,
                            'xmin': 0.375,
                            'xmax': model_radius,
                            'input_concentration': 2.15E5,
                            },
                        }
    test_feces_contamination.recharge_parameters = recharge_parameters
    test_feces_contamination.ibound_parameters.pop("top_boundary1")
    # Add well as ibound
    test_feces_contamination.ibound_parameters['well1'] = {'head': 0.0,
                                                'top': -30.0,
                                                'bot': -50.0,
                                                'xmin': 0.0,
                                                'xmax': 0.1,
                                                'ibound': -1}


    # print(test_phrea.__dict__)
    modpath_phrea = mpw.ModPathWell(test_feces_contamination, #test_phrea,
                            workspace = workspace,
                            modelname = "semiconfined",
                            bound_left = "xmin",
                            bound_right = "xmax")
    # print(modpath_phrea.__dict__)

    # Run phreatic schematisation
    modpath_phrea.run_model(run_mfmodel = True,
                        run_mpmodel = True)


    # Pollutant to define removal parameters for
    MS2_organism = TR.MicrobialOrganism(organism_name=organism_name,
                                        alpha0_suboxic = alpha0["suboxic"],
                                        alpha0_anoxic = alpha0["anoxic"],
                                        alpha0_deeply_anoxic =alpha0["deeply_anoxic"],
                                        pH0_suboxic =pH0["suboxic"],
                                        pH0_anoxic =pH0["anoxic"],
                                        pH0_deeply_anoxic = pH0["deeply_anoxic"],
                                        mu1_suboxic = mu1["suboxic"],
                                        mu1_anoxic = mu1["anoxic"],
                                        mu1_deeply_anoxic = mu1["deeply_anoxic"],
                                        organism_diam = organism_diam
                                        )

    # Removal parameters organism
    rem_parms = MS2_organism.organism_dict

    # Calculate advective microbial removal
    modpath_transport = TR.Transport(modpath_phrea,
                            pollutant = MS2_organism)

    # Calculate advective microbial removal
    # Final concentration per endpoint_id
    C_final = {}
    # df_particle before advective removal
    df_particle = modpath_transport.df_particle
    df_flowline = modpath_transport.df_flowline

    for endpoint_id in ["well1"]: #modpath_phrea.schematisation_dict.get("endpoint_id"):
        df_particle, df_flowline, C_final[endpoint_id] = modpath_transport.calc_advective_microbial_removal(
                                            df_particle = df_particle, df_flowline = df_flowline,
                                            endpoint_id = endpoint_id,
                                            trackingdirection = modpath_phrea.trackingdirection)

    # Add final conc to df_output
    df_output.loc[df_output.Scen_nr == iScen,"C_final"] = C_final["well1"]
    # df_output
    output_fname = os.path.join(modpath_phrea.dstroot,f"df_output_scen{iScen}.csv")
    df_output.to_csv(output_fname)

d:\Sutra2_tool\sutra2\testing\test_mpw_mbo_defecation_human_noleak_diffuse_sensitivity
Run phreatic model
Run model: d:\Sutra2_tool\sutra2\testing\test_mpw_mbo_defecation_human_noleak_diffuse_sensitivity semiconfined

FloPy is using the following executable to run the model: .\mf2005.exe

                                  MODFLOW-2005     
    U.S. GEOLOGICAL SURVEY MODULAR FINITE-DIFFERENCE GROUND-WATER FLOW MODEL
                             Version 1.9.01 5/01/2012                        

 Using NAME file: semiconfined.nam 
 Run start date and time (yyyy/mm/dd hh:mm:ss): 2022/08/12 10:31:27

 Solving:  Stress period:     1    Time step:     1    Ground-Water Flow Eqn.
 Run end date and time (yyyy/mm/dd hh:mm:ss): 2022/08/12 10:31:27
 Elapsed run time:  0.017 Seconds

  Normal termination of simulation
Model run d:\Sutra2_tool\sutra2\testing\test_mpw_mbo_defecation_human_noleak_diffuse_sensitivity semiconfined completed without errors: True
Run modpath: d:\Sutra2_tool\sutra2\testing